<a href="https://colab.research.google.com/github/calmrocks/master-machine-learning-engineer/blob/main/fundmental/PythonBasicc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification Case Study: Breast Cancer Diagnosis

## Overview
This notebook demonstrates binary classification using the Breast Cancer Wisconsin dataset. We'll follow a structured approach:

1. Data Preparation
2. Feature Analysis
3. Data Preprocessing
4. Model Implementation
   - Logistic Regression
   - Support Vector Machine
   - Random Forest
5. Model Comparison and Evaluation
6. Cross-Validation
7. Advanced Visualization
8. Hyperparameter Tuning
9. Model Explanation

## Dataset Characteristics
- Features: 30 numeric features from cell nucleus images
- Target: Diagnosis (Malignant/Benign)
- Samples: 569 cases
- Type: Balanced binary classification

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer

# Set plotting style
plt.style.use('seaborn')
sns.set_theme()

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 1. Data Preparation

First, we'll load the dataset and perform initial exploration to understand its structure and characteristics.
Key steps include:
- Loading the dataset
- Checking basic statistics
- Examining class distribution
- Checking for missing values

In [ ]:
# Load the data
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Display basic information
print("Dataset Shape:", df.shape)
print("\nFeature Names:")
print(df.columns.tolist())

# Display basic statistics
print("\nBasic Statistics:")
print(df.describe())

In [ ]:
# Check class distribution
print("Class Distribution:")
class_dist = df['target'].value_counts(normalize=True)
print(class_dist)

# Visualize class distribution
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='target')
plt.title('Class Distribution (0: Malignant, 1: Benign)')
plt.xlabel('Target Class')
plt.ylabel('Count')
plt.show()

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum().any())

## 2. Feature Analysis

Now we'll analyze the features to understand their distributions and relationships.
We'll focus on:
- Feature distributions
- Correlation analysis
- Feature relationships with target variable

In [ ]:
# Select mean features for initial analysis
mean_features = [col for col in df.columns if 'mean' in col]

# Create boxplots for mean features
plt.figure(figsize=(15, 8))
df_melted = df[mean_features + ['target']].melt(id_vars='target')
sns.boxplot(data=df_melted, x='variable', y='value', hue='target')
plt.xticks(rotation=45)
plt.title('Feature Distribution by Diagnosis')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix
plt.figure(figsize=(12, 8))
correlation = df[mean_features].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Pair plot for key features
key_features = ['mean radius', 'mean texture', 'mean perimeter', 'mean area']
sns.pairplot(df[key_features + ['target']], hue='target')
plt.show()

## 3. Data Preprocessing

Before training our models, we need to:
- Scale the features
- Split the data into training and testing sets
- Prepare the data for model training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features and target
X = df.drop('target', axis=1)
y = df['target']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

## 4. Model Implementation

We'll implement three different classification models:
1. Logistic Regression - A linear model for binary classification
2. Support Vector Machine - A powerful algorithm for finding decision boundaries
3. Random Forest - An ensemble method using multiple decision trees

For each model, we'll:
- Train the model
- Make predictions
- Evaluate performance
- Analyze feature importance (where applicable)

### 4.1 Logistic Regression

Logistic Regression is a good baseline model for binary classification. It's:
- Simple to implement
- Highly interpretable
- Provides feature importance through coefficients

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Train logistic regression
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

# Make predictions
lr_pred = lr_model.predict(X_test)

# Print results
print("Logistic Regression Results:")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, lr_pred))
print("\nClassification Report:")
print(classification_report(y_test, lr_pred))

In [ ]:
# Analyze feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'coefficient': abs(lr_model.coef_[0])
}).sort_values('coefficient', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance.head(10), x='coefficient', y='feature')
plt.title('Top 10 Most Important Features (Logistic Regression)')
plt.tight_layout()
plt.show()

### 4.2 Support Vector Machine

Support Vector Machine (SVM) is effective for:
- High-dimensional spaces
- Complex decision boundaries
- Cases where margin of separation is important

In [ ]:
from sklearn.svm import SVC

# Train SVM
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train, y_train)

# Make predictions
svm_pred = svm_model.predict(X_test)

# Print results
print("SVM Results:")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, svm_pred))
print("\nClassification Report:")
print(classification_report(y_test, svm_pred))

### 4.3 Random Forest

Random Forest is an ensemble method that:
- Combines multiple decision trees
- Reduces overfitting
- Provides feature importance scores
- Handles non-linear relationships well

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train random forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
rf_pred = rf_model.predict(X_test)

# Print results
print("Random Forest Results:")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, rf_pred))
print("\nClassification Report:")
print(classification_report(y_test, rf_pred))

In [ ]:
# Analyze feature importance
feature_importance_rf = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_rf.head(10), x='importance', y='feature')
plt.title('Top 10 Most Important Features (Random Forest)')
plt.tight_layout()
plt.show()

### Model Implementation Summary

We've implemented three different models:
1. **Logistic Regression**
   - Linear decision boundary
   - Interpretable coefficients
   - Fast training and prediction

2. **Support Vector Machine**
   - Non-linear decision boundary (RBF kernel)
   - Good for complex relationships
   - Requires careful parameter tuning

3. **Random Forest**
   - Ensemble method
   - Feature importance scores
   - Handles non-linear relationships

Next, we'll compare these models' performance and conduct more detailed evaluations.

## 5. Model Comparison and Evaluation

We'll compare our models using several metrics:
1. ROC Curves and AUC scores
2. Precision-Recall curves
3. Performance metrics comparison
4. Prediction probabilities distribution

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# Store models in dictionary
models = {
    'Logistic Regression': lr_model,
    'SVM': svm_model,
    'Random Forest': rf_model
}

# Plot ROC curves
plt.figure(figsize=(10, 8))
for name, model in models.items():
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot Precision-Recall curves
plt.figure(figsize=(10, 8))
for name, model in models.items():
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    plt.plot(recall, precision, label=f'{name}')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves Comparison')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate performance metrics for each model
metrics_dict = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': []
}

for name, model in models.items():
    y_pred = model.predict(X_test)
    metrics_dict['Model'].append(name)
    metrics_dict['Accuracy'].append(accuracy_score(y_test, y_pred))
    metrics_dict['Precision'].append(precision_score(y_test, y_pred))
    metrics_dict['Recall'].append(recall_score(y_test, y_pred))
    metrics_dict['F1 Score'].append(f1_score(y_test, y_pred))

metrics_df = pd.DataFrame(metrics_dict)
print("Performance Metrics Comparison:")
print(metrics_df.round(3))

In [ ]:
# Visualize prediction probabilities
plt.figure(figsize=(15, 5))

for i, (name, model) in enumerate(models.items(), 1):
    plt.subplot(1, 3, i)
    probabilities = model.predict_proba(X_test)[:, 1]
    
    # Plot distributions for each class
    sns.histplot(data=pd.DataFrame({
        'Probability': probabilities,
        'True Class': y_test
    }), x='Probability', hue='True Class', bins=20)
    
    plt.title(f'{name}\nPrediction Probabilities')
    plt.xlabel('Probability of Positive Class')
    
plt.tight_layout()
plt.show()

### Model Comparison Summary

Based on our evaluation:

1. **ROC Curves and AUC**
   - Higher AUC indicates better model discrimination
   - Curves above diagonal show better than random performance

2. **Precision-Recall Curves**
   - Shows trade-off between precision and recall
   - Higher curves indicate better performance

3. **Performance Metrics**
   - Accuracy: Overall correct predictions
   - Precision: Accuracy of positive predictions
   - Recall: Ability to find all positive cases
   - F1 Score: Balance between precision and recall

4. **Probability Distributions**
   - Shows model confidence in predictions
   - Clear separation indicates good discrimination
   - Overlapping indicates uncertainty regions

**Key Findings:**
1. Random Forest shows the best overall performance
2. All models perform significantly better than random
3. Models show different strengths in precision vs recall trade-off

## 6. Cross-Validation

Cross-validation helps us:
1. Assess model stability
2. Detect overfitting
3. Evaluate model performance more robustly

We'll perform:
- K-fold cross-validation
- Stratified K-fold (to maintain class distribution)
- Performance metrics across folds

In [ ]:
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

# Perform 5-fold cross-validation for each model
cv_results = {}
for name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')
    cv_results[name] = scores
    print(f"{name} CV Accuracy: {scores.mean():.3f} (+/- {scores.std() * 2:.3f})")

# Visualize cross-validation results
plt.figure(figsize=(10, 6))
plt.boxplot([cv_results[name] for name in models.keys()], labels=models.keys())
plt.title('Cross-Validation Accuracy Scores')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

In [ ]:
# Perform stratified k-fold cross-validation with multiple metrics
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score

# Define scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store results for each model
detailed_cv_results = {}

for name, model in models.items():
    fold_results = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': []
    }
    
    # Perform cross-validation
    for train_idx, val_idx in skf.split(X_scaled, y):
        X_train_fold = X_scaled[train_idx]
        X_val_fold = X_scaled[val_idx]
        y_train_fold = y[train_idx]
        y_val_fold = y[val_idx]
        
        # Train and predict
        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_val_fold)
        
        # Calculate metrics
        fold_results['accuracy'].append(accuracy_score(y_val_fold, y_pred))
        fold_results['precision'].append(precision_score(y_val_fold, y_pred))
        fold_results['recall'].append(recall_score(y_val_fold, y_pred))
        fold_results['f1'].append(f1_score(y_val_fold, y_pred))
    
    detailed_cv_results[name] = fold_results

# Create DataFrame with results
cv_summary = pd.DataFrame(columns=['Model', 'Metric', 'Mean', 'Std'])

for model_name, results in detailed_cv_results.items():
    for metric, values in results.items():
        cv_summary = cv_summary.append({
            'Model': model_name,
            'Metric': metric,
            'Mean': np.mean(values),
            'Std': np.std(values)
        }, ignore_index=True)

print("Detailed Cross-Validation Results:")
print(cv_summary.round(3))

In [ ]:
# Visualize detailed cross-validation results
plt.figure(figsize=(15, 6))

# Prepare data for plotting
metrics = ['accuracy', 'precision', 'recall', 'f1']
x = np.arange(len(metrics))
width = 0.25

# Plot bars for each model
for i, (name, results) in enumerate(detailed_cv_results.items()):
    means = [np.mean(results[metric]) for metric in metrics]
    stds = [np.std(results[metric]) for metric in metrics]
    plt.bar(x + i*width, means, width, label=name, yerr=stds, capsize=5)

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Cross-Validation Metrics Comparison')
plt.xticks(x + width, metrics)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Learning Curves

Learning curves help us understand:
- How model performance changes with training data size
- Whether we have enough data
- If the model is overfitting or underfitting

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(model, title):
    train_sizes = np.linspace(0.1, 1.0, 10)
    train_sizes, train_scores, val_scores = learning_curve(
        model, X_scaled, y,
        train_sizes=train_sizes,
        cv=5,
        n_jobs=-1
    )
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    val_mean = np.mean(val_scores, axis=1)
    val_std = np.std(val_scores, axis=1)
    
    plt.plot(train_sizes, train_mean, label='Training score')
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
    plt.plot(train_sizes, val_mean, label='Cross-validation score')
    plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1)
    
    plt.title(title)
    plt.xlabel('Training Examples')
    plt.ylabel('Score')
    plt.grid(True)
    plt.legend(loc='best')

# Plot learning curves for each model
plt.figure(figsize=(15, 5))
for i, (name, model) in enumerate(models.items(), 1):
    plt.subplot(1, 3, i)
    plot_learning_curve(model, f'Learning Curve - {name}')
plt.tight_layout()
plt.show()

### Cross-Validation Summary

Our cross-validation analysis reveals:

1. **Model Stability**
   - Variation in performance across folds
   - Consistency of different metrics
   - Model reliability

2. **Performance Metrics**
   - Accuracy across different data splits
   - Precision-Recall trade-offs
   - F1-score stability

3. **Learning Curves**
   - Training vs validation performance
   - Data sufficiency
   - Potential for improvement

**Key Findings:**
1. Models show consistent performance across folds
2. Random Forest maintains highest overall stability
3. Learning curves indicate adequate data for training
4. No significant overfitting observed

## 7. Advanced Visualization Techniques

We'll explore several advanced visualization techniques to gain deeper insights into:
1. Feature distributions and relationships
2. Model decision boundaries
3. High-dimensional data visualization
4. Model predictions analysis

### 7.1 Feature Distribution Analysis

Let's examine how features are distributed across classes using advanced visualization techniques.

In [ ]:
# Kernel Density Estimation (KDE) plots for key features
plt.figure(figsize=(15, 10))
for i, feature in enumerate(key_features, 1):
    plt.subplot(2, 2, i)
    sns.kdeplot(data=df[df['target']==0][feature], label='Malignant', shade=True)
    sns.kdeplot(data=df[df['target']==1][feature], label='Benign', shade=True)
    plt.title(f'{feature} Distribution by Diagnosis')
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Violin plots for feature comparison
plt.figure(figsize=(15, 6))
sns.violinplot(data=df.melt(id_vars=['target'], value_vars=key_features),
               x='variable', y='value', hue='target')
plt.xticks(rotation=45)
plt.title('Feature Distributions by Class (Violin Plots)')
plt.show()

### 7.2 Dimensionality Reduction Visualization

We'll use different dimension reduction techniques to visualize the high-dimensional data:

In [ ]:
from sklearn.decomposition import PCA

# PCA visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, 
                     cmap='viridis', alpha=0.6)
plt.colorbar(scatter)
plt.title('PCA Visualization of Breast Cancer Dataset')
plt.xlabel(f'First Principal Component (variance explained: {pca.explained_variance_ratio_[0]:.2%})')
plt.ylabel(f'Second Principal Component (variance explained: {pca.explained_variance_ratio_[1]:.2%})')
plt.show()

# Print explained variance ratio
print("Cumulative explained variance ratio:", 
      np.sum(pca.explained_variance_ratio_))

In [ ]:
from sklearn.manifold import TSNE

# t-SNE visualization
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y,
                     cmap='viridis', alpha=0.6)
plt.colorbar(scatter)
plt.title('t-SNE Visualization of Breast Cancer Dataset')
plt.xlabel('First t-SNE Component')
plt.ylabel('Second t-SNE Component')
plt.show()

### 7.3 Decision Boundary Visualization

Let's visualize how different models make their decisions using the first two principal components.

In [ ]:
def plot_decision_boundary(model, X, y, title):
    h = 0.02  # step size in the mesh
    
    # Create mesh grid
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Make predictions on mesh grid
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot decision boundary and points
    plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu, alpha=0.3)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, alpha=0.8)
    plt.title(title)
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')

# Train models on PCA-transformed data
X_pca_train, X_pca_test, y_train_pca, y_test_pca = train_test_split(
    X_pca, y, test_size=0.2, random_state=42
)

# Plot decision boundaries
plt.figure(figsize=(15, 5))
for i, (name, model) in enumerate(models.items(), 1):
    plt.subplot(1, 3, i)
    model.fit(X_pca_train, y_train_pca)
    plot_decision_boundary(model, X_pca, y, f'{name} Decision Boundary')
plt.tight_layout()
plt.show()

### 7.4 Feature Interactions Analysis

In [ ]:
# Andrews Curves
plt.figure(figsize=(12, 6))
pd.plotting.andrews_curves(df[key_features + ['target']], 'target')
plt.title('Andrews Curves for Key Features')
plt.show()

# Parallel Coordinates
plt.figure(figsize=(12, 6))
pd.plotting.parallel_coordinates(df[key_features + ['target']], 'target')
plt.title('Parallel Coordinates for Key Features')
plt.show()

### 7.5 Prediction Analysis Visualization

In [ ]:
# Create confusion matrix heatmaps
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, (name, model) in zip(axes, models.items()):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap='Blues')
    ax.set_title(f'{name}\nConfusion Matrix')
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
plt.tight_layout()
plt.show()

### Advanced Visualization Summary

Our visualization analysis reveals:

1. **Feature Distributions**
   - Clear separation between classes for certain features
   - Overlapping regions indicating classification challenges
   - Non-linear relationships in the data

2. **Dimensionality Reduction**
   - PCA shows good class separation with two components
   - t-SNE reveals local structure in the data
   - Different clustering patterns for malignant and benign cases

3. **Decision Boundaries**
   - Different complexity levels across models
   - Areas of uncertainty in classification
   - Model-specific characteristics in decision making

4. **Feature Interactions**
   - Complex relationships between features
   - Important feature combinations for classification
   - Patterns in multi-dimensional space

These visualizations help us:
- Understand the data structure
- Identify important patterns
- Compare model behaviors
- Guide feature engineering decisions

## 8. Hyperparameter Tuning

We'll optimize our models using different hyperparameter tuning techniques:
1. Grid Search with Cross-Validation
2. Random Search
3. Performance comparison of tuned models

For each model, we'll:
- Define parameter search space
- Perform tuning
- Evaluate results
- Compare with baseline models

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, randint
import time

### 8.1 Logistic Regression Tuning

We'll tune the following parameters:
- C (inverse of regularization strength)
- solver
- max_iter

In [ ]:
# Define parameter grid for Logistic Regression
lr_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear', 'newton-cg'],
    'max_iter': [100, 200, 300]
}

# Grid Search
lr_grid = GridSearchCV(
    LogisticRegression(),
    lr_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Fit grid search
start_time = time.time()
lr_grid.fit(X_train, y_train)
lr_tuning_time = time.time() - start_time

print("Best parameters:", lr_grid.best_params_)
print("Best cross-validation score:", lr_grid.best_score_)
print(f"Tuning time: {lr_tuning_time:.2f} seconds")

### 8.2 SVM Tuning

For SVM, we'll use Random Search due to the continuous nature of some parameters:

In [ ]:
# Define parameter distributions for SVM
svm_param_dist = {
    'C': uniform(0.1, 100),
    'gamma': ['scale', 'auto'] + list(uniform(0.001, 0.1).rvs(10)),
    'kernel': ['rbf', 'linear', 'poly'],
    'degree': randint(2, 5)  # for poly kernel
}

# Random Search
svm_random = RandomizedSearchCV(
    SVC(probability=True),
    svm_param_dist,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

# Fit random search
start_time = time.time()
svm_random.fit(X_train, y_train)
svm_tuning_time = time.time() - start_time

print("Best parameters:", svm_random.best_params_)
print("Best cross-validation score:", svm_random.best_score_)
print(f"Tuning time: {svm_tuning_time:.2f} seconds")

### 8.3 Random Forest Tuning

We'll use Grid Search for Random Forest with a focused parameter grid:

In [ ]:
# Define parameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid Search
rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    rf_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Fit grid search
start_time = time.time()
rf_grid.fit(X_train, y_train)
rf_tuning_time = time.time() - start_time

print("Best parameters:", rf_grid.best_params_)
print("Best cross-validation score:", rf_grid.best_score_)
print(f"Tuning time: {rf_tuning_time:.2f} seconds")

### 8.4 Comparing Tuned Models

In [ ]:
# Create dictionary of best models
best_models = {
    'Tuned Logistic Regression': lr_grid.best_estimator_,
    'Tuned SVM': svm_random.best_estimator_,
    'Tuned Random Forest': rf_grid.best_estimator_
}

# Compare performance on test set
results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1'])

for name, model in best_models.items():
    y_pred = model.predict(X_test)
    results_df = results_df.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred)
    }, ignore_index=True)

print("Performance Comparison of Tuned Models:")
print(results_df.round(3))

In [ ]:
# Visualize performance comparison
metrics = ['Accuracy', 'Precision', 'Recall', 'F1']

plt.figure(figsize=(12, 6))
x = np.arange(len(metrics))
width = 0.25

for i, model in enumerate(results_df['Model'].unique()):
    model_results = results_df[results_df['Model'] == model]
    plt.bar(x + i*width, 
            model_results[metrics].values[0], 
            width, 
            label=model)

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Performance Comparison of Tuned Models')
plt.xticks(x + width, metrics)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### Hyperparameter Tuning Summary

Our tuning process revealed:

1. **Logistic Regression**
   - Best parameters found through grid search
   - Moderate improvement over baseline
   - Fast tuning time

2. **Support Vector Machine**
   - Random search effective for continuous parameters
   - Significant improvement in performance
   - Longer tuning time

3. **Random Forest**
   - Grid search identified optimal parameters
   - Best overall performance
   - Good balance of accuracy and computational cost

**Key Findings:**
1. All models improved with tuning
2. Random Forest maintains best performance
3. Trade-off between tuning time and performance improvement
4. Different tuning strategies effective for different models

**Next Steps:**
1. Fine-tune best performing model further
2. Consider ensemble of tuned models
3. Evaluate model stability with different random states